In [1]:
#include <iostream>
#include <string>

using namespace std;

# Class templates
Just like we can create function templates, we can also create class templates, allowing classes to have members that use template parameters as types. For example: 

In [2]:
template <class T>
class mypair {
    T values [2];
  public:
    mypair (T first, T second)
    {
      values[0]=first; values[1]=second;
    }
};

The class that we have just defined serves to store two elements of any valid type. For example, if we wanted to declare an object of this class to store two integer values of type int with the values 115 and 36 we would write:

In [3]:
mypair<int> myobject (115, 36);

This same class could also be used to create an object to store any other type, such as:

In [4]:
mypair<double> myfloats (3.0, 2.18);

The constructor is the only member function in the previous class template and it has been defined inline within the class definition itself. In case that a member function is defined outside the defintion of the class template, it shall be preceded with the template <...> prefix:

In [6]:
template <class T>
class mypair {
    T a, b;
  public:
    mypair (T first, T second) {a=first; b=second;}
    T getmax ();
};

In [7]:
template <class T>
T mypair<T>::getmax ()
{
  return a>b? a : b;
}

In [8]:
mypair <int> myobject (100, 75);
cout << myobject.getmax();

100

Notice the syntax of the definition of member function getmax:
```cpp
template <class T>
T mypair<T>::getmax ()
```
Confused by so many T's? There are three T's in this declaration: The first one is the template parameter. The second T refers to the type returned by the function. And the third T (the one between angle brackets) is also a requirement: It specifies that this function's template parameter is also the class template parameter.

## Template specialization
It is possible to define a different implementation for a template when a specific type is passed as template argument. This is called a template specialization.
For example, let's suppose that we have a very simple class called mycontainer that can store one element of any type and that has just one member function called increase, which increases its value. But we find that when it stores an element of type char it would be more convenient to have a completely different implementation with a function member uppercase, so we decide to declare a class template specialization for that type:

In [5]:
// class template:
template <class T>
class mycontainer {
    T element;
  public:
    mycontainer (T arg) {element=arg;}
    T increase () {return ++element;}
};

In [6]:
// class template specialization:
template <>
class mycontainer <char> {
    char element;
  public:
    mycontainer (char arg) {element=arg;}
    char uppercase ()
    {
      if ((element>='a')&&(element<='z'))
      element+='A'-'a';
      return element;
    }
};

In [8]:
mycontainer<int> myint (7);
mycontainer<char> mychar ('h');

cout << myint.increase() << endl;
cout << mychar.uppercase() << endl;

8
H


This is the syntax used for the class template specialization:
```cpp
template <> class mycontainer <char> { ... };
```
First of all, notice that we precede the class name with template<> , including an empty parameter list. This is because all types are known and no template arguments are required for this specialization, but still, it is the specialization of a class template, and thus it requires to be noted as such.
But more important than this prefix, is the <char> specialization parameter after the class template name. This specialization parameter itself identifies the type for which the template class is being specialized (char). Notice the differences between the generic class template and the specialization:
```cpp    
template <class T> class mycontainer { ... };

template <> class mycontainer <char> { ... };
```
The first line is the generic template, and the second one is the specialization.
When we declare specializations for a template class, we must also define all its members, even those identical to the generic template class, because there is no "inheritance" of members from the generic template to the specialization.

# Special members
Special member functions are member functions that are implicitly defined as member of classes under certain circumstances. There are six:

| Member function | typical form for class C: |
| -- | -- |
| Default constructor | C::C(); |
| Destructor | C::~C(); |
| Copy constructor | C::C (const C&); |
| Copy assignment | C& operator= (const C&); |
| Move constructor | C::C (C&&); |
| Move assignment | C& operator= (C&&); |

Let's examine each of these:

## Default constructor
The default constructor is the constructor called when objects of a class are declared, but are not initialized with any arguments.
If a class definition has no constructors, the compiler assumes the class to have an implicitly defined default constructor. Therefore, after declaring a class like this:
```cpp
class Example {
  public:
    int total;
    void accumulate (int x) { total += x; }
};
```
The compiler assumes that Example has a default constructor. Therefore, objects of this class can be constructed by simply declaring them without any arguments:
```cpp
Example ex;
```
But as soon as a class has some constructor taking any number of parameters explicitly declared, the compiler no longer provides an implicit default constructor, and no longer allows the declaration of new objects of that class without arguments. For example, the following class:
```cpp
class Example2 {
  public:
    int total;
    Example2 (int initial_value) : total(initial_value) { };
    void accumulate (int x) { total += x; };
};
```
Here, we have declared a constructor with a parameter of type int. Therefore the following object declaration would be correct:
```cpp
Example2 ex (100);   // ok: calls constructor
```
But the following:
```cpp
Example2 ex;         // not valid: no default constructor 
```
Would not be valid, since the class has been declared with an explicit constructor taking one argument and that replaces the implicit default constructor taking none.
Therefore, if objects of this class need to be constructed without arguments, the proper default constructor shall also be declared in the class. For example:

In [15]:
class Example3 {
    string data;
  public:
    Example3 (const string& str) : data(str) {}
    Example3() {}
    const string& content() const {return data;}
};

In [16]:
Example3 foo;
Example3 bar ("Example");

cout << "bar's content: " << bar.content() << '\n';

bar's content: Example


Here, Example3 has a default constructor (i.e., a constructor without parameters) defined as an empty block:
```cpp
Example3() {}
```
This allows objects of class Example3 to be constructed without arguments (like foo was declared in this example). Normally, a default constructor like this is implicitly defined for all classes that have no other constructors and thus no explicit definition is required. But in this case, Example3 has another constructor:
```cpp
Example3 (const string& str);
```
And when any constructor is explicitly declared in a class, no implicit default constructors is automatically provided.

## Destructor
Destructors fulfill the opposite functionality of constructors: They are responsible for the necessary cleanup needed by a class when its lifetime ends. The classes we have defined in previous chapters did not allocate any resource and thus did not really require any clean up.
But now, let's imagine that the class in the last example allocates dynamic memory to store the string it had as data member; in this case, it would be very useful to have a function called automatically at the end of the object's life in charge of releasing this memory. To do this, we use a destructor. A destructor is a member function very similar to a default constructor: it takes no arguments and returns nothing, not even void. It also uses the class name as its own name, but preceded with a tilde sign (~):

In [17]:
class Example4 {
    string* ptr;
  public:
    // constructors:
    Example4() : ptr(new string) {}
    Example4 (const string& str) : ptr(new string(str)) {}
    // destructor:
    ~Example4 () {delete ptr;}
    // access content:
    const string& content() const {return *ptr;}
};

In [18]:
Example4 foo;
Example4 bar ("Example");

cout << "bar's content: " << bar.content() << '\n';

bar's content: Example


On construction, Example4 allocates storage for a string. Storage that is later released by the destructor.
The destructor for an object is called at the end of its lifetime; in the case of foo and bar this happens at the end of function main.

## Copy constructor
When an object is passed a named object of its own type as argument, its copy constructor is invoked in order to construct a copy.
A copy constructor is a constructor whose first parameter is of type reference to the class itself (possibly const qualified) and which can be invoked with a single argument of this type. For example, for a class MyClass, the copy constructor may have the following signature:
MyClass::MyClass (const MyClass&);
If a class has no custom copy nor move constructors (or assignments) defined, an implicit copy constructor is provided. This copy constructor simply performs a copy of its own members. For example, for a class such as:
```cpp
class MyClass {
  public:
    int a, b; string c;
};
```

An implicit copy constructor is automatically defined. The definition assumed for this function performs a shallow copy, roughly equivalent to:
```cpp
MyClass::MyClass(const MyClass& x) : a(x.a), b(x.b), c(x.c) {}
```
This default copy constructor may suit the needs of many classes. But shallow copies only copy the members of the class themselves, and this is probably not what we expect for classes like class Example4 we defined above, because it contains pointers of which it handles its storage. For that class, performing a shallow copy means that the pointer value is copied, but not the content itself; This means that both objects (the copy and the original) would be sharing a single string object (they would both be pointing to the same object), and at some point (on destruction) both objects would try to delete the same block of memory, probably causing the program to crash on runtime. This can be solved by defining the following custom copy constructor that performs a deep copy:

In [19]:
class Example5 {
    string* ptr;
  public:
    Example5 (const string& str) : ptr(new string(str)) {}
    ~Example5 () {delete ptr;}
    // copy constructor:
    Example5 (const Example5& x) : ptr(new string(x.content())) {}
    // access content:
    const string& content() const {return *ptr;}
};

In [20]:
Example5 foo ("Example");
Example5 bar = foo;

cout << "bar's content: " << bar.content() << '\n';

bar's content: Example


The deep copy performed by this copy constructor allocates storage for a new string, which is initialized to contain a copy of the original object. In this way, both objects (copy and original) have distinct copies of the content stored in different locations.